In [31]:
from matminer.datasets import load_dataset

# dielectric = load_dataset("matbench_dielectric")
# log_gvrh = load_dataset("matbench_log_gvrh")
# jdft2d = load_dataset("matbench_jdft2d")

boltztrap_mp = load_dataset("boltztrap_mp")

# matbench_dielectric
matbench_expt_gap
# matbench_expt_is_metal
# matbench_glass
# matbench_jdft2d
# matbench_steels

Fetching boltztrap_mp.json.gz from https://ndownloader.figshare.com/files/13297511 to /Users/muon/opt/anaconda3/lib/python3.9/site-packages/matminer/datasets/boltztrap_mp.json.gz


Fetching https://ndownloader.figshare.com/files/13297511 in MB: 14.020608MB [00:00, 363.05MB/s]                         


In [41]:
import pandas as pd
# dielectric.iloc[0].structure 
# boltztrap_mp.columns

# expt_gap.composition

pd.Series(list(set(boltztrap_mp.formula) & set(expt_is_metal.composition)))


0        SrCrO4
1          PrF3
2           SnS
3      Tl9SbSe6
4       K4GeSe4
         ...   
362         TlS
363      CrPbO4
364    LiNiP2O7
365         LiF
366          KF
Length: 367, dtype: object

In [48]:
boltztrap_mp.head()

,mpid,pf_n,pf_p,s_n,s_p,formula,m_n,m_p,structure
0,mp-10070,0.865,0.0125,-421.0,760.0,BaAg(PO3)3,0.578,72.800,"[[ 0.08245398 10.58009491 11.61923254] O, [3.1..."
1,mp-10086,1.050,0.6440,-393.0,567.0,YSF,0.541,4.020,"[[2.84699546 0.94899849 0. ] F, [0.9489..."
2,mp-10096,0.977,0.4200,-305.0,471.0,Na3Sr3GaP4,0.270,1.020,"[[-2.8085287 7.06608376 1.25800196] Na, [1...."
3,mp-10103,0.889,0.0257,-416.0,871.0,YbLiAlF6,0.548,20.300,"[[2.52287112 1.45658029 7.18290524] Li, [-2.85..."
4,mp-10155,0.719,1.5700,-496.0,510.0,P2Ir,1.160,0.667,"[[0.86371064 5.14343422 4.80450637] P, [4.5512..."
